# MakerSpace Jam

In [ ]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Primary Dependencies and Context Setting

In [ ]:
!pip3 install -U -q openai==0.27.8 llama-index==0.8.6 nltk==3.8.1 python-dotenv

### Load the OPENAI API key

In [ ]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

### Load token list

In [ ]:
import pandas as pd

sheet_name = "GSW_token_list"
workbook_id = '1MB1ZsQul4AB262AsaY4fHtGW4HWp2-56zB-E5xTbs2A'
url = f'https://docs.google.com/spreadsheets/d/{workbook_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df = pd.read_csv(url)
df.head()

In [ ]:
token_list = df['Token'].tolist()

In [ ]:
token_string = '_'.join(token_list)
token_string

## Data Loading

### Webpage

Using the Trafilatura Web Reader.

In [ ]:
!pip3 install trafilatura

In [ ]:
# list of webpages to index
webpages = [ "https://www.basketball-reference.com/" ]

In [ ]:
from llama_index import TrafilaturaWebReader

web_docs = TrafilaturaWebReader().load_data([webpages[0]])
web_docs[0].text

In [ ]:
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(
    chunk_size=1000 ### YOUR CODE HERE
)

node_parser = SimpleNodeParser(
    text_splitter=text_splitter ### YOUR CODE HERE
)

# parse nodes from workbooks and insert into vector index
for w_doc in web_docs:
    nodes = node_parser.get_nodes_from_documents([w_doc])
    for node in nodes:
        node.metadata = {'title': 'Basketball Stats and History',
                         'type': 'webpage',
                         'url': 'https://www.basketball-reference.com/',
                         'description': 'Basketball Stats and History',
                         'accessibility': 'public'
                        }

### Email Samples

In [ ]:
from llama_index import Document
from llama_index.schema import MetadataMode

In [135]:
docEmailSample = Document(
    text="Hey KD, let's grab dinner after the game on October 24th, Steph", 
    metadata={
        'from_to': 'Stephen Curry / Kevin Durant',
        # 'datetime': "2023-09-01T15:30:00Z",
        # 'email_from': 'kevin.durant@suns.nba',
        # 'email_to': 'stephen.curry@warriors.nba',
        # "token_list": token_string,
        # 'from': 'Stephen Curry',
        # 'to': 'Kevin Durant'
    }
)
docEmailSample2 = Document(
    text="Yo Joker, you were a monster last year, can't wait to play against you in the opener! Draymond", 
    metadata={
        'from_to': 'Draymond Green / Nikola Jokic',
        # 'datetime': "2023-09-01T02:30:00Z",
        # 'email_from': 'lebron.james@lakers.nba',
        # 'email_to': 'nikola.jokic@nuggets.nba',
        # "token_list": token_string,
        # 'team': 'Golden State Warriors',
        # 'from': 'Draymond Green',
        # 'to': 'Nikola Jokic'
    }
)
docScheduleSample = Document(
    text="""
        {
            "Date": "Tue Oct 24 2023",
            "Start (ET)": "7:30p",
            "Visitor/Neutral": "Golden State Warriors",
            "PTS": "",
            "Home/Neutral": "Denver Nuggets",
            "PTS": "",
            "Attend.": "",
            "Arena": "Ball Arena",
            "Notes": ""
        }
    """,
    metadata={
        'team': 'Golden State Warriors',
        # "token_list": token_string,
        # 'teams': 'Golden State Warriors' # idk how to get list of metadata
    }
)
docScheduleSample2 = Document(
    text="""
        {
            "Date": "Fri Oct 27 2023",
            "Start (ET)": "10:00p",
            "Visitor/Neutral": "Phoenix Suns",
            "PTS": "",
            "Home/Neutral": "Golden State Warriors",
            "PTS": "",
            "Attend.": "",
            "Arena": "Chase Center",
            "Notes": ""
        }
    """,
    metadata={
        'team': 'Golden State Warriors',     
        # "token_list": token_string,
        # 'team': 'Golden State Warriors' # idk how to get list of metadata
############################ YOU ASK(??) FOR SMTH LIKE THIS:
        # 'list': [
        #     {'team': 'warriors'}, 
        #     {'team': 'lakers'},
        #     {'team': 'nuggets'},
        # ]
    }
)

docAdditionalSamples = [docEmailSample, docEmailSample2, docScheduleSample, docScheduleSample2]

## Indexing

### Context setting

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()
chunk_size = 1000
llm = OpenAI(
    temperature=0, 
    model="gpt-3.5-turbo",
    streaming=True
)

service_context = ServiceContext.from_defaults(
    llm=llm, 
    chunk_size=chunk_size,
    embed_model=embed_model
)

### Vector Store

This is intended to be a global vector store to insert the nodes from all data sources

In [ ]:
!pip3 install -U -q chromadb==0.4.6 tiktoken==0.4.0 sentence-transformers==2.2.2 pydantic==1.10.11

In [ ]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
import chromadb

In [136]:
chroma_client = chromadb.Client()
# chroma_collection = chroma_client.create_collection("all_data")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context, service_context=service_context)

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


### Populate Vector Store with Nodes

Parse nodes for each loaded data source and insert it to the vector store.

In [137]:
# vector_index.insert_nodes(nodes) 
# vector_index.insert_nodes(docExamplesToPlayWith)
vector_index.insert_nodes(docAdditionalSamples)

### Setup Metadata Filtering

In [138]:
from llama_index.tools import FunctionTool
from llama_index.vector_stores.types import (
    VectorStoreInfo,
    MetadataInfo,
    ExactMatchFilter,
    MetadataFilters,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

from typing import List, Tuple, Any
from pydantic import BaseModel, Field

top_k = 3

info_emails_players = VectorStoreInfo(
    content_info="emails exchanged between NBA players",
    metadata_info=[
        MetadataInfo(
            name="from_to",
            type="str",
            description="""
email sent by a player of the Golden State Warriors to any other NBA player, one of [
Stephen Curry / any NBA player, 
Klay Thompson / any NBA player, 
Chris Paul / any NBA player, 
Andrew Wiggins / any NBA player, 
Draymond Green / any NBA player, 
Gary Payton II / any NBA player, 
Kevon Looney / any NBA player, 
Jonathan Kuminga / any NBA player, 
Moses Moody / any NBA player, 
Brandin Podziemski / any NBA player, 
Cory Joseph / any NBA player, 
Dario Šarić / any NBA player]"""
        ), 
        # MetadataInfo(
        #     name="to",
        #     type="str",
        #     description="""receiver of the email text, can be any NBA player"""
        # ), 
        # MetadataInfo(
        #     name='doctype',
        #     type="str",
        #     description="email"
        # )
    ]
)
info_schedule_team = VectorStoreInfo(
    content_info="schedule of the Golden State Warriors games",
    metadata_info=[
        MetadataInfo(
            name="team",
            type="str",
            description="Golden State Warriors"
        ), 
        # MetadataInfo(
        #     name='doctype',
        #     type="str",
        #     description="schedule"
        # )
    ]
)

In [139]:
class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="natural language query string")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[str] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names specified in filter_key_list)"
        )
    )

In [141]:
def auto_retrieve_fn(
    query: str, filter_key_list: List[str], filter_value_list: List[str]
):
    """Auto retrieval function.

    Performs auto-retrieval from a vector database, and then applies a set of filters.

    """
    query = query or "Query"
    
    # for i, (k, v) in enumerate(zip(filter_key_list, filter_value_list)):
    #     if k == 'token_list':
    #         if token not in v:
    #             v = ''

    exact_match_filters = [
        ExactMatchFilter(key=k, value=v)
        for k, v in zip(filter_key_list, filter_value_list)
    ]
    retriever = VectorIndexRetriever(
        vector_index, filters=MetadataFilters(filters=exact_match_filters), top_k=top_k
    )
    # query_engine = vector_index.as_query_engine(filters=MetadataFilters(filters=exact_match_filters))
    query_engine = RetrieverQueryEngine.from_args(retriever)

    response = query_engine.query(query)
    return str(response)

In [143]:
description_emails = f"""\
Use this tool to look up information about emails exchanged betweed players of the Golden State Warriors and any other NBA player.
The vector database schema is given below:
{info_emails_players.json()}
"""
auto_retrieve_tool_emails = FunctionTool.from_defaults(
    fn=auto_retrieve_fn, 
    name='auto_retrieve_tool_emails',
    description=description_emails, 
    fn_schema=AutoRetrieveModel
)

description_schedule = f"""\
Use this tool to look up the NBA game schedule of the Golden State Warriors.
The vector database schema is given below:
{info_schedule_team.json()}
"""
auto_retrieve_tool_schedule = FunctionTool.from_defaults(
    fn=auto_retrieve_fn, 
    name='auto_retrieve_tool_schedule',
    description=description_schedule, 
    fn_schema=AutoRetrieveModel
)



## SQL NBA Query Tool

In [ ]:
!pip3 install -q -U pandas

In [ ]:
import pandas as pd

def get_df_from_workbook(sheet_name,
                         workbook_id = '1MB1ZsQul4AB262AsaY4fHtGW4HWp2-56zB-E5xTbs2A'):
    url = f'https://docs.google.com/spreadsheets/d/{workbook_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    return pd.read_csv(url)

In [ ]:
sheet_names = ['Teams', 'Players', 'Schedule', 'Player_Stats']
dict_of_dfs = {sheet: get_df_from_workbook(sheet) for sheet in sheet_names}
dict_of_dfs['Teams'].head()

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:")

In [ ]:
for dict in dict_of_dfs:
    print(dict) 

In [ ]:
dict_of_dfs['Teams'].head()

In [ ]:
for df in dict_of_dfs:
    dict_of_dfs[df].to_sql(df, con=engine)

In [ ]:
from llama_index import SQLDatabase

sql_database = SQLDatabase(
    engine,
    # include_tables=['Teams', 'Players_2023-24', 'Schedule_2023-24', 'Player_Stats_2022-23_(Playoffs)', 'Player_Stats_2022-23_(Regular_Season)']
    include_tables=list(dict_of_dfs.keys())
    )

In [ ]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=list(dict_of_dfs.keys())
)

In [ ]:
from llama_index.tools.query_engine import QueryEngineTool

sql_nba_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine, # 
    name='sql_nba_tool', 
    description=("""Useful for translating a natural language query into a SQL query over tables containing:
                    1. teams, containing information related to all NBA teams
                    2. players, containing information related to all NBA players and the teams they play for
                    3. schedule, containing information related to the entire NBA game schedule
                    4. player_stats_playoffs, containing information related to all NBA player stats
                    """
    ),

)

In [144]:
from llama_index.agent import OpenAIAgent

agent = OpenAIAgent.from_tools(
    [sql_nba_tool, 
     auto_retrieve_tool_emails,
     auto_retrieve_tool_schedule], 
    llm=llm, verbose=True 
)

In [146]:
response = agent.chat("I am Stephen Curry. Do I have an email from Kevin Durant? When are we playing against his team?")

=== Calling Function ===
Calling function: auto_retrieve_tool_emails with args: {
  "query": "Stephen Curry / Kevin Durant",
  "filter_key_list": ["from_to"],
  "filter_value_list": ["Stephen Curry / Kevin Durant"]
}
Got output: Stephen Curry is suggesting to Kevin Durant that they grab dinner after the game on October 24th.
=== Calling Function ===
Calling function: auto_retrieve_tool_schedule with args: {
  "query": "Golden State Warriors"
}


TypeError: auto_retrieve_fn() missing 2 required positional arguments: 'filter_key_list' and 'filter_value_list'

## SQL GSW Query Tool

In [ ]:
sheet_names = ['GSW_Team_Roster', 'GSW_Players_Summary', 'GSW_staff_and_executives']
dict_of_dfs = {sheet: get_df_from_workbook(sheet) for sheet in sheet_names}
dict_of_dfs['GSW_Team_Roster'].head()